In [1]:
from mlx_lm import load, generate

model, tokenizer = load("mlx-community/Llama-3.2-3B-Instruct-4bit")

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [2]:
import pandas as pd
import json
from fuzzywuzzy import process


company_names = {
    "ucb": "UCB.BR",
    "abinbev": "ABI.BR",
    "aedifica": "AED.BR",
    "ageas": "AGS.BR",
    "aperam": "APAM.AS",
    "argenx": "ARGX.BR",
    "belgian national bank": "BNB.BR",
    "cofinimmo": "COFB.BR",
    "elia": "ELI.BR",
    "ishares msci belgium etf": "EWK",
    "groupe bruxelles lambert sa": "GBLB.BR",
    "galapagos nv": "GLPG.AS",
    "kbc bank": "KBC.BR",
    "melexis nv": "MELE.BR",
    "proximus": "PROX.BR",
    "sofina": "SOF.BR",
    "solvay": "SOLB.BR",
    "umicore": "UMI.BR",
    "unified post group": "UPG.BR",
    "warehouses de pauw": "WDP.BR",
    "xior student housing": "XIOR.BR",
}

companies = list(company_names.keys())
# Threshold for acceptable similarity (0-100 scale)
threshold = 80


def find_best_match(user_input, company_list, threshold):
    # Find the best match with score
    best_match, best_score = process.extractOne(user_input, company_list)

    # Check if the best match meets the threshold
    if best_score >= threshold:
        return best_match
    else:
        return "none"


def read_file_to_string(file_name):
    try:
        with open(file_name, "r", encoding="utf-8") as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return "Error: File not found."
    except IOError:
        return "Error: Could not read the file."


def retrieve_stock_etf_info(companies: list, stock_name: str, threshold: int) -> str:
    stock_name = stock_name.lower()
    result = find_best_match(stock_name, companies, threshold)
    # let's do a fuzzy match with the company names and retrieve the most relevant company name
    res = ""
    if result != "none":
        file_content = read_file_to_string("data/" + company_names[result] + ".txt")
        res = json.dumps({"information we have on stock/etf": file_content})
    else:
        res = json.dumps({"error": "transaction id not found."})
    return res

/Users/naimsassine/anaconda3/envs/mistral_ai/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
user_input = "what is the latest trading price of the UCB stock?"

In [4]:
# Format message with the command-r tool use template
messages = [
    {
        "role": "user",
        "content": user_input,
    }
]
# Define tools available for the model to use:
tools = [
    {
        "name": "retrieve_stock_etf_info",
        "description": "get info about a certain stock or etf given the name of the instrument",
        "parameter_definitions": {
            "stock_name": {
                "description": "Name of the stock or ETF for which we need information",
                "type": "str",
                "required": True,
            }
        },
    },
    {
        "name": "directly_answer",
        "description": "Calls a standard (un-augmented) AI chatbot to generate a response given the conversation history",
        "parameter_definitions": {},
    },
]

formatted_input = tokenizer.apply_chat_template(
    messages, tools=tools, tokenize=False, add_generation_prompt=True
)
response = generate(model, tokenizer, prompt=formatted_input, verbose=False)

In [5]:
print(response)

{"name": "retrieve_stock_etf_info", "parameters": {"stock_name": "UCB"}}<|eom_id|><|start_header_id|>assistant<|end_header_id|>

<|python_tag|>{"name": "retrieve_stock_etf_info", "parameters": {"stock_name": "UCB"}}<|eom_id|><|start_header_id|>assistant<|end_header_id|>

The function "retrieve_stock_etf_info" requires a string argument "stock_name" to get the latest trading price of the UCB stock.


In [6]:
import re

json_match = re.search(r"\{.*\}", response)
if json_match:
    json_string = json_match.group()
    data = json.loads(json_string)
    print(data)
else:
    print("No JSON found")

{'name': 'retrieve_stock_etf_info', 'parameters': {'stock_name': 'UCB'}}


In [ ]:
import functools

names_to_functions = {
    "retrieve_stock_etf_info": functools.partial(
        retrieve_stock_etf_info, companies=companies, threshold=80
    ),
}

# to be called after the tool answer
function_name = data["name"]
function_params = data["parameters"]
print("\nfunction_name: ", function_name, "\nfunction_params: ", function_params)

function_result = names_to_functions[function_name](**function_params)
function_result

In [ ]:
formatted_input = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

response = generate(model, tokenizer, prompt=formatted_input, verbose=False)

In [ ]:
print(response)

In [ ]:
messages